In [35]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

![Crisp_DM](../reports\figures\Crisp_DM.png)

#   Data Understanding 

* RKI ,webscaraping https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* john Hopkin (Github)https://github.com/CSSEGISandData/COVID-19.git
* rest API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/


In [36]:
git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [37]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [38]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [39]:
pd_raw=pd.read_csv(data_path)

In [40]:
pd_raw

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/17/22,6/18/22,6/19/22,6/20/22,6/21/22,6/22/22,6/23/22,6/24/22,6/25/22,6/26/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,181534,181574,181666,181725,181808,181912,181987,182033,182072,182149
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,277141,277141,277409,277444,277663,277940,278211,278504,278793,279077
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265968,265971,265975,265985,265993,266006,266015,266025,266030,266038
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,43449,43449,43449,43449,43449,43774,43774,43774,43774,43774
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761
5,NaN,Antarctica,-71.949900,23.347000,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
6,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,8555,8581,8581,8581,8581,8590,8590,8625,8625,8625
7,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,9313453,9313453,9341492,9341492,9341492,9341492,9341492,9341492,9341492,9367172
8,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,423044,423044,423044,423104,423104,423104,423104,423104,423104,423104
9,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,145457,146264,147096,147942,148996,150123,151113,152199,153012,153939


In [41]:
import requests
from bs4 import BeautifulSoup

In [42]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [43]:
soup = BeautifulSoup(page.content,'html.parser')

In [44]:
html_table = soup.find('table')

In [45]:
all_rows =html_table.find_all('tr')

In [46]:
final_data_list = []

In [47]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [48]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [49]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,3.830.851,15.741,56.487,"508,8",16.294
3,Bayern,5.118.697,20.424,74.116,"564,0",24.354
4,Berlin,1.096.712,4.181,15.775,"430,5",4.640
5,Branden­burg,819.092,2.630,10.211,"403,4",5.713
6,Bremen,215.074,1.142,5.379,"790,9",789


#  REST API Call

In [50]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronafälle_in_den_Bundesländern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [51]:
import json

In [52]:
json_object=json.loads(data.content) 

In [53]:
type(json_object)

dict

In [54]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [55]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [56]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,821782,1656367200000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,28231.442436,4.573731e+10,2.881496e+06,2597,895.022974,26053,2,"895,0",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,628232,1656367200000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,33913.061316,2.089396e+09,4.188002e+05,2721,707.214877,13101,3,"707,2",2
2,3,03,Niedersachsen,Land,8003421,9,2594609,1656367200000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,32418.749432,1.299836e+11,4.008988e+06,9622,921.455962,73748,3,"921,5",3
3,4,04,Bremen,Freie Hansestadt,680130,5,215074,1656367200000,04,4132268b-54de-4327-ac1e-760e915112f1,31622.483937,1.119157e+09,3.357177e+05,789,790.878214,5379,2,"790,9",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5683816,1656367200000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,31707.867588,8.782936e+10,2.648673e+06,25716,774.000492,138744,11,"774,0",5


# API access via REST service, e.g. USA data

example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [57]:
import requests